In [1]:
import torch
import torch.nn as nn

In [ ]:
class Actor(nn.Module):
    def __init__(self, input_features, num_action):
        super().__init__()
        self.actor = nn.Sequential(
            nn.Linear(input_features, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_action),
            nn.softmax(dim = -1)
        )

    def forward(self, state):
        x = self.actor(state)
        return x

In [7]:
class Critic(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.critic = nn.Sequential(
            nn.Linear(input_features, 128),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, state):
        x = self.critic(state)
        return x

In [8]:
def init_weight(model):
    for layers in model.modules():
        nn.init.kaiming_normal_(layers.weight, mode='fan_in', nonlinearity='relu')
        if layers.bias is not None:
            nn.init.zeros_(layers.bias)

In [ ]:
class PPO_Agent():
    def __init__(self, learning_rate):
        self.device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.A_model = Actor(2, 4).to(self.device)
        self.C_model = Critic(2).to(self.device)
        self.A_opt   = torch.optim.Adam(self.A_model.parameters(), lr = learning_rate)
        self.A_ex_model = Actor(2, 4).to(device)
        self.C_opt   = torch.optim.Adam(self.C_model.parameters(), lr = learning_rate)
        self.A_ex_model.load_state_dict(self.A_model.state_dict())
        self.returns = {}
    
    def get_episode(self):
        


    def train(self, state):
        action_probs = self.A_model(state)
        dist = Categorical(action_probs)
        action_logprobs = dist.log_prob(action)
        disct_entropy = dist.entropy()
        state_values = self.C_model(state)
        rewards = []
        discounted_reward = 0
        g = 0
        for state, action, reward in reversed(episode):
            self.returns[state] = reward + self.gamma * g
            return torch
        for epoch in range(num_epochs):
            new_action_probs = self.A_model
            expected_state_values = self.C_model(state)
            advantage = (predicted_state_values - expected_state_values)**2
            self.C_opt.zero_grad()
            advantage.backward()
            self.C_opt.step()

            with torch.no_grad():
                predicted_state_values = self.C_model(state)
            advantage = (predicted_state_values.detach() - expected_state_values)**2

            ratio = (action_logprobs / action_old_logprobs) * expected_state_values

            self.A_opt.zero_grad()
            ratio.backward()




        
        


In [1]:
alpha = {}

In [2]:
type(alpha)

dict

In [3]:
alpha[1] = 'rahul kumar'

In [4]:
print(alpha)

{1: 'rahul kumar'}


In [ ]:
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

################################## set device ##################################
print("============================================================================================")
# set device to cpu or cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device is set as : {device}")
print("============================================================================================")


################################## PPO Policy ##################################
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []
        self.is_terminals = []
    
    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.is_terminals[:]

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space):
        super().__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space :
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Tanh()
                        )
        else:
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )
            
    def forward(self, x):
        return self.actor(x)

class Critic(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.Tanh(),
                        nn.Linear(64, 64),
                        nn.Tanh(),
                        nn.Linear(64, 1)
                    )
        
    def forward(self, state):
        return self.critic(state)
        
class ActorCritic(nn.Module):
    def __init__(self,state_dim, action_dim, alpha, beta):
        super().__init__()
        self.has_continuous_action_space = alpha
        self.action_dim = action_dim
        self.actornet = Actor(state_dim, action_dim, alpha).to(device)
        self.criticnet = Critic(state_dim).to(device)

    def act(self, state):

        if self.has_continuous_action_space:
            with torch.no_grad():
                action_mean = self.actornet(state)
            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0)
            dist = MultivariateNormal(action_mean, cov_mat)
        else:
            with torch.no_grad():
                action_probs = self.actornet(state)
            dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        with torch.no_grad():
            state_val = self.criticnet(state)

        return action, action_logprob, state_val
    
    def evaluate(self, state, action):

        if self.has_continuous_action_space:
            action_mean = self.actornet(state)
            
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var).to(device)
            dist = MultivariateNormal(action_mean, cov_mat)
            
            # For Single Action Environments.
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)
        else:
            action_probs = self.actornet(state)
            dist = Categorical(action_probs)
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.criticnet(state)
        
        return action_logprobs, state_values, dist_entropy
    
    def set_action_std(self, new_action_std):
        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")



class PPO(nn.Module):
    def __init__(self, state_dim =4, action_dim=2, lr_actor=0.003, lr_critic=0.001, gamma=0.2, K_epochs=80, eps_clip=0.2, has_continuous_action_space = True, action_std_init=0.6):
        super().__init__()
        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init)
        self.A_opt = torch.optim.Adam(self.policy.actornet.parameters(), lr = lr_actor)
        self.C_opt = torch.optim.Adam(self.policy.criticnet.parameters(), lr = lr_critic)
        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init)
        self.policy_old.actornet.load_state_dict(self.policy.actornet.state_dict())
        self.policy_old.criticnet.load_state_dict(self.policy.criticnet.state_dict())
        
        self.MseLoss = nn.MSELoss()

    def set_action_std(self, new_action_std):
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")

    def decay_action_std(self, action_std_decay_rate, min_action_std):
        print("--------------------------------------------------------------------------------------------")
        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")
        print("--------------------------------------------------------------------------------------------")

    def select_action(self, state):

        if self.has_continuous_action_space:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)

            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.detach().cpu().numpy().flatten()
        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)
            
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.item()

    def update(self):
        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_state_values = torch.squeeze(torch.stack(self.buffer.state_values, dim=0)).detach().to(device)

        # calculate advantages
        advantages = rewards - old_state_values.detach()

        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss  
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) - 0.01 * dist_entropy
            loss_1 = self.MseLoss(state_values, rewards)
            
            # take gradient step
            self.A_opt.zero_grad()
            loss.mean().backward()
            self.A_opt.step()

            self.C_opt.zero_grad()
            loss_1.mean().backward()
            self.C_opt.step()
            
        # Copy new weights into old policy
        self.policy_old.actornet.load_state_dict(self.policy.actornet.state_dict())
        self.policy_old.criticnet.load_state_dict(self.policy.criticnet.state_dict())

        # clear buffer
        self.buffer.clear()
        
        
       

Device is set as : cpu


In [10]:
model = PPO()

Actor(
  (actor): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=2, bias=True)
    (5): Tanh()
  )
)
Critic(
  (critic): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [15]:
def decorator_function(f):
    def wrapper(a, b, c):
        print(f"befor {c}")
        result = f(a,b)
        print(f"after {c}")
        return result
    
    return wrapper

@decorator_function
def funct(a, b):
    return a + b

print(funct(3, 5, 9))


befor 9
after 9
8


In [16]:
list = [2,5,2,4,1]

In [17]:
b = sorted(list)

In [ ]:
print(list)
print(b)
list.sort()


[2, 5, 2, 4, 1]
[1, 2, 2, 4, 5]


In [20]:
print(list)

[1, 2, 2, 4, 5]


In [21]:
max(list)

5

In [22]:
min(list)

1

In [23]:
sum(list)

14

In [32]:
import torch

x = torch.tensor(5, dtype = torch.float32)
x.requires_grad = True

In [50]:
y = (x**2).detach()

dy_dx = torch.autograd.grad(y, x, create_graph=True, grad_outputs=torch.ones_like(x))
d2y_dx2 = torch.autograd.grad(dy_dx, x , grad_outputs = torch.ones_like(x))

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [46]:
print(d2y_dx2)

(tensor(2.),)


In [51]:
print(y.requires_grad)

False


In [52]:
print(x.requires_grad)

True


In [87]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1,1),
        )

    def forward(self, x):
        return self.layer(x)

In [88]:
model_1 = NeuralNet()
model_2 = NeuralNet()

In [110]:
a = model_1(torch.tensor([[2]],dtype = torch.float32))

b = model_2(a.detach())

loss_funct = nn.MSELoss()
loss = loss_funct(b, torch.tensor([[1]], dtype = torch.float32))
loss.backward()

b_1 = model_2(a)

loss = loss_funct(b_1, torch.tensor([[1]], dtype = torch.float32))
loss.backward()

In [108]:
b_1 = model_2(a)

In [109]:
loss = loss_funct(b_1, torch.tensor([[1]], dtype = torch.float32))
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [91]:
for name, param in model_1.named_parameters():
    if param.requires_grad:
        print(name, param.shape)


layer.0.weight torch.Size([1, 1])
layer.0.bias torch.Size([1])


In [85]:
loss = a - b

In [86]:
loss.backward()

In [84]:
print(a)
print(b)

tensor([[0.1495]], grad_fn=<AddmmBackward0>)
tensor([[0.3474]], grad_fn=<AddmmBackward0>)


In [65]:
print(a.requires_grad)

False


In [72]:
b = model(torch.tensor([[5]],dtype = torch.float32))

In [73]:
print(b.requires_grad)

True


In [74]:
c = (b - a)**2

In [75]:
print(c.requires_grad)

True


In [76]:
x = torch.tensor(2, dtype = torch.float32)
y = torch.tensor(4, dtype = torch.float32)


In [77]:
x.requires_grad = True
y.requires_grad = False

In [92]:
z = x**2 + y**2
z = z.detach()

In [97]:
n = z*x

In [98]:
print(n.requires_grad)

True


In [93]:
dz_dx = torch.autograd.grad(z, x, create_graph = True, grad_outputs = torch.ones_like(x))

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [80]:
print(dz_dx)

(tensor(4., grad_fn=<MulBackward0>),)
